# Nova captura RJ: [Painel Covid](http://painel.saude.rj.gov.br/monitoramento/covid19.html#)

- Atualiza o CSV todo dia às 17h em: http://painel.saude.rj.gov.br/arquivos/COVID.CSV

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

## Importa dados

- Dados coletados do painel:

In [3]:
df = pd.read_csv("http://painel.saude.rj.gov.br/arquivos/COVID.CSV", sep=";")
df.info()

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452758 entries, 0 to 452757
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   sexo                452758 non-null  object 
 1   idade               431064 non-null  float64
 2   municipio_res       452758 non-null  object 
 3   uf                  452758 non-null  object 
 4   dt_sintoma          452283 non-null  object 
 5   dt_coleta_dt_notif  452162 non-null  object 
 6   classificacao       452758 non-null  object 
 7   evolucao            26292 non-null   object 
 8   dt_obito            26203 non-null   object 
 9   comorbidade         0 non-null       float64
 10  dias                452338 non-null  float64
 11  dt_evento           452338 non-null  object 
 12  bairro              245541 non-null  object 
 13  cep                 236728 non-null  object 
dtypes: float64(3), object(11)
memory usage: 48.4+ MB


In [4]:
df.head()

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta_dt_notif,classificacao,evolucao,dt_obito,comorbidade,dias,dt_evento,bairro,cep
0,F,46.0,NOVA IGUACU ...,RJ,2020/05/05 00:00:00.000,2020/05/05 00:00:00.000,CONFIRMADO,NaN,NaN,NaN,248.0,2020/05/05 00:00:00.000,NaN,26000000
1,M,23.0,MACAE ...,RJ,2020/05/03 00:00:00.000,2020/05/06 00:00:00.000,CONFIRMADO,NaN,NaN,NaN,250.0,2020/05/03 00:00:00.000,PARQUE AEROPORTO ...,NaN
2,M,23.0,MACAE ...,RJ,2020/05/01 00:00:00.000,2020/05/03 00:00:00.000,CONFIRMADO,NaN,NaN,NaN,252.0,2020/05/01 00:00:00.000,AJUDA DE BAIXO ...,27910000
3,M,55.0,ANGRA DOS REIS ...,RJ,2020/05/23 00:00:00.000,2020/05/23 00:00:00.000,CONFIRMADO,NaN,NaN,NaN,230.0,2020/05/23 00:00:00.000,CENTRO ...,23900000
4,M,53.0,MESQUITA ...,RJ,2020/10/04 00:00:00.000,2020/10/13 00:00:00.000,CONFIRMADO,NaN,NaN,NaN,96.0,2020/10/04 00:00:00.000,COREIA ...,26556005


- Dados presentes no [Brasil.io](https://brasil.io/dataset/covid19/caso_full/) para o dia 30/12:

_Publicados em https://coronavirus.rj.gov.br/boletim/boletim-coronavirus-30-12-25-303-obitos-e-428-373-casos-confirmados-no-rj/_

In [9]:
compare = pd.read_excel("../raw/casos-RJ-2020-12-30-fernanda.scovino-1.xlsx")
compare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   municipio    94 non-null     object
 1   confirmados  94 non-null     int64 
 2   mortes       94 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ KB


In [10]:
compare.head()

,municipio,confirmados,mortes
0,RIO DE JANEIRO,164782,14743
1,NITEROI,22764,690
2,SAO GONCALO,18090,993
3,BELFORD ROXO,13498,374
4,DUQUE DE CAXIAS,13118,908


## Descobrindo a data de confirmados e mortes

Colunas cadidatas para casos confirmados:
- `dt_evento`
- `dt_coleta_dt_notif`
- `dt_sintoma`

Colunas cadidatas para obitos confirmados:
- `dt_obito`

In [5]:
# Remove espaco no final
df["classificacao"] = df["classificacao"].str.strip()
df["municipio_res"] = df["municipio_res"].str.strip()

# Converte datas
datecols = ["dt_sintoma", "dt_obito", "dt_evento", "dt_coleta_dt_notif"]
for col in datecols:
    df[col] = pd.to_datetime(df[col])

# Testando colunas candidadtas para casos & obitos confirmados até 30/12/20
date = "2020-12-30"
test = pd.DataFrame()
for col in datecols:
    test[col] = df[(df[col] < date) & (df["classificacao"] == "CONFIRMADO")].reset_index().groupby("municipio_res")["index"].count()

test.loc["TOTAL NO ESTADO"] = test.sum(axis=0)

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93 entries, ANGRA DOS REIS to TOTAL NO ESTADO
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   dt_sintoma          93 non-null     int64
 1   dt_obito            93 non-null     int64
 2   dt_evento           93 non-null     int64
 3   dt_coleta_dt_notif  93 non-null     int64
dtypes: int64(4)
memory usage: 3.6+ KB


### Compara resultados com o divulgado

In [12]:
test = compare.set_index("municipio").join(test)
test

,confirmados,mortes,dt_sintoma,dt_obito,dt_evento,dt_coleta_dt_notif
municipio,,,,,,
RIO DE JANEIRO,164782,14743,166986.0,15211.0,167013.0,170806.0
NITEROI,22764,690,23991.0,706.0,23994.0,23883.0
SAO GONCALO,18090,993,18556.0,1037.0,18556.0,18440.0
BELFORD ROXO,13498,374,14638.0,378.0,14638.0,14570.0
DUQUE DE CAXIAS,13118,908,13465.0,922.0,13465.0,13326.0
...,...,...,...,...,...,...
SAO SEBASTIAO DO ALTO,158,6,183.0,6.0,183.0,174.0
DUAS BARRAS,107,5,110.0,5.0,110.0,108.0
RIO DAS FLORES,35,5,35.0,5.0,35.0,35.0


In [15]:
test.to_csv("../outputs/br_rj_check_painel_covid_2020_12_30.csv")